In [1]:
import os
import pandas as pd
#list the files
filelist = os.listdir('inspectbot-data/') 
#read them into pandas
train_df = pd.DataFrame(filelist)

In [2]:
import librosa
import glob
import random
import librosa.display
import numpy as np
import soundfile as sf
# import librosa.output
import re


In [3]:
train_df.size


124

In [4]:
train_df = train_df.rename(columns={0:'file'})



In [5]:
train_df

,file
0,MM-1644214.wav
1,OO-1655290.wav
2,MM-1642188.wav
3,OO-1655291.wav
4,MM-1642181.wav
...,...
119,OO-1652387.wav
120,OO-16523812.wav
121,OO-16482313.wav
122,OO-1648230.wav


In [6]:
Command = []
for i in range(0, len(train_df)):
  Command.append(''.join(re.split("[^a-zA-Z]*",train_df['file'][i].split('-')[0])))

In [7]:
train_df['command'] = Command


In [8]:
train_df

,file,command
0,MM-1644214.wav,MM
1,OO-1655290.wav,OO
2,MM-1642188.wav,MM
3,OO-1655291.wav,OO
4,MM-1642181.wav,MM
...,...,...
119,OO-1652387.wav,OO
120,OO-16523812.wav,OO
121,OO-16482313.wav,OO
122,OO-1648230.wav,OO


In [9]:
def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
    file_name = os.path.join(os.path.abspath('inspectbot-data')+'/'+str(files.file))

    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name,sr=22000) 

    # # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    # mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    # stft = np.abs(librosa.stft(X))

    # # Computes a chromagram from a waveform or power spectrogram.
    # chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # # Computes a mel-scaled spectrogram.
    # mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # # Computes spectral contrast
    # contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # # Computes the tonal centroid features (tonnetz)
    # tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    # sr=sample_rate).T,axis=0)

    return X

In [10]:
train_features = train_df.apply(extract_features, axis=1)
# 


In [11]:
len(train_features)

124

In [12]:
# train_features =extract_features(train_df.iloc[:, :])

In [13]:
features_train = []
for i in range(0, 124):
    features_train.append(train_features[i])
    

In [14]:
len(features_train[0])

21967

In [22]:
X_train = np.array(librosa.util.normalize(features_train,axis=0))

In [23]:
X_train[0]

array([0.        , 0.        , 0.        , ..., 0.01797722, 0.02093433,
       0.        ], dtype=float32)

In [24]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical


In [25]:
y_train = np.array(train_df['command'])

In [26]:
# y_train

In [27]:
len(X_train )

124

In [28]:
for j in range(25):
  for i,sample in enumerate(X_train):
    if i <124:
      n = random. randint(0,9) 

      noise = np.random.normal(0, n/100, sample.shape)
      # print(noise)
      new=sample+noise

      X_train=np.append(X_train, [new], axis= 0 )
      y_train=np.append(y_train,y_train[i])
    else :
      break

  # df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'), index=['x', 'y'])

In [33]:

len(X_train[3])

21967

In [34]:
for i,sample in enumerate(X_train):
  save_path = 'inspectbot-augmented'
  filename=y_train[i]+'-'+str(i)+'.wav'
  completeName = os. path. join(save_path, filename)
  sf.write(completeName, sample, 21967)

In [27]:
#  len(os.listdir('new') )

In [28]:
# !zip -r ./Augmented.zip ./new/


In [29]:
# from google.colab import files
# files.download("/content/Augmented.zip")

In [30]:
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
# y_val = to_categorical(lb.fit_transform(y_val))

In [31]:
from sklearn.preprocessing import StandardScaler


In [ ]:
y_train=y_train.reshape(3339,1,1,8)

ValueError: cannot reshape array of size 248 into shape (3339,1,1,8)

In [ ]:
# ss = StandardScaler()
# X_train = ss.fit_transform(X_train)
# X_val = ss.transform(X_val)
# X_test = ss.transform(X_test)


In [44]:
X_train.shape

(3339, 9985)

In [46]:
 X_train=X_train.reshape(3339,9985,1,1)

In [47]:
,9985,1,1)# X_train.shape

In [74]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [65]:
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, GlobalAveragePooling2D, Reshape, GlobalAveragePooling3D
# from keras.callbacks import EarlyStopping

In [81]:

model = Sequential()

model.add(Conv2D(32,kernel_size=(1,5), strides=2,activation = 'relu',padding="valid",input_shape=( 9985, 1, 1)))

model.add(Conv2D(32,kernel_size=(1,5), strides=1,activation = 'relu',padding="valid"))
model.add(Conv2D(32,kernel_size=(1,5), strides=2,activation = 'relu',padding="valid"))

model.add(Conv2D(32,kernel_size=(1,5), strides=1,activation = 'relu',padding="valid"))

model.add(Conv2D(32,kernel_size=(1,5), strides=2,activation = 'relu',padding="valid"))

model.add(Conv2D(32,kernel_size=(1,5), strides=1,activation = 'relu',padding="valid"))

model.add(Conv2D(64,kernel_size=(1,5), strides=2,activation = 'relu',padding="valid"))
model.add(Conv2D(64,kernel_size=(1,5), strides=1,activation = 'relu',padding="valid"))

model.add(Conv2D(128,kernel_size=(1,5), strides=2,activation = 'relu',padding="valid"))

model.add(Conv2D(128,kernel_size=(1,5), strides=1,activation = 'relu',padding="valid"))

model.add(GlobalAveragePooling2D())
model.add(Reshape((1,1,128))) 
model.add(Conv2D(8,kernel_size=(1,1), strides=1,activation = 'softmax',padding="valid"))






model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='sgd')
model.summary()



ValueError: Negative dimension size caused by subtracting 5 from 1 for '{{node conv2d_40/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_40/Conv2D/ReadVariableOp)' with input shapes: [?,9985,1,1], [1,5,1,32].

In [76]:
!source /.bashrc

In [77]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GT 730, pci bus id: 0000:01:00.0, compute capability: 3.5



In [80]:
history = model.fit(X_train, y_train, batch_size=500, epochs=100, 
                    validation_split = 0.2
                    )

Epoch 1/100
6/6 [==============================] - 17s 3s/step - loss: 2.0794 - accuracy: 0.1295 - val_loss: 2.0796 - val_accuracy: 0.1108
Epoch 2/100
6/6 [==============================] - 17s 3s/step - loss: 2.0794 - accuracy: 0.1295 - val_loss: 2.0797 - val_accuracy: 0.1108
Epoch 3/100
6/6 [==============================] - 17s 3s/step - loss: 2.0793 - accuracy: 0.1295 - val_loss: 2.0797 - val_accuracy: 0.1108
Epoch 4/100
6/6 [==============================] - 17s 3s/step - loss: 2.0793 - accuracy: 0.1295 - val_loss: 2.0797 - val_accuracy: 0.1108
Epoch 5/100
1/6 [====>.........................] - ETA: 14s - loss: 2.0794 - accuracy: 0.1360

KeyboardInterrupt: 

In [ ]:
!nvidia-smi